Setup

In [3]:
#Download kernel data
!mkdir data
!kaggle datasets download neuromusic/avocado-prices -p ./data

  0%|                                                | 0.00/629k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 629k/629k [00:00<00:00, 27.8MB/s]


In [2]:
#Libraries
import pandas as pd
import numpy as np
import re as re

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [37]:
df_avocado = pd.read_csv('data/avocado.csv')

In [38]:
df_avocado.drop(list(df_avocado)[0], axis = 1, inplace = True)
df_avocado.drop(labels = 'year', axis = 1, inplace = True)

#convert to snake case
df_avocado.columns = [i.replace(" ", "_") for i in df_avocado.columns]
df_avocado.columns = [re.sub("(?<=[a-z])(?=[A-Z])", "_", i) for i in df_avocado.columns]
df_avocado.columns = [i.lower() for i in df_avocado.columns]

EDA

In [5]:
df_avocado.head(5)

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [6]:
df_avocado.dtypes

date              object
average_price    float64
total_volume     float64
4046             float64
4225             float64
4770             float64
total_bags       float64
small_bags       float64
large_bags       float64
xlarge_bags      float64
type              object
year               int64
region            object
dtype: object

In [40]:
df_avocado['avo_type_sum'] = df_avocado['4046'] + df_avocado['4225'] + df_avocado['4770']

In [ ]:
#Sometimes the total volume differs

In [41]:
total_test = df_avocado['avo_type_sum'] + df_avocado['total_bags'] == df_avocado['total_volume']

In [9]:
len(total_test)

18249

In [10]:
sum(total_test)

12879

In [42]:
#Add seasonal variables
add_datepart(df_avocado, 'date', drop = False)

In [43]:
#number of unique values, missing values
df_avocado.columns = [i.replace(" ", "_") for i in df_avocado.columns]
df_avocado.columns = [re.sub("(?<=[a-z])(?=[A-Z])", "_", i) for i in df_avocado.columns]
df_avocado.columns = [i.lower() for i in df_avocado.columns]


Modelling

In [44]:
df_avocado.columns

Index(['date', 'average_price', 'total_volume', '4046', '4225', '4770',
       'total_bags', 'small_bags', 'large_bags', 'xlarge_bags', 'type',
       'region', 'avo_type_sum', 'year', 'month', 'week', 'day', 'dayofweek',
       'dayofyear', 'is_month_end', 'is_month_start', 'is_quarter_end',
       'is_quarter_start', 'is_year_end', 'is_year_start', 'elapsed'],
      dtype='object')

In [46]:
factor_vars = ['type', 'region', 'year', 'month', 'week', 'day', 'dayofweek',
       'dayofyear', 'is_month_end', 'is_month_start', 'is_quarter_end',
       'is_quarter_start', 'is_year_end', 'is_year_start']

#*** up to
for f in factor_vars:

In [47]:
[(c, len(df_avocado[c].cat.categories)+1) for c in factor_vars]

AttributeError: Can only use .cat accessor with a 'category' dtype